<h1 style="text-align: center;">MIS 542 &ndash; Advanced Business Analytics</h1>
<h1 style="text-align: center;">Third Business Case - Fund Raising</h1>

<h4> Team 7 Members - Section 1:
<h4> Nada Alomari - 202111090 | Saja Alqahtani - 202111070 | Reda Alhamza - 201365390

In [1]:
import pandas as pd

fund_df = pd.read_csv('D:/MIS542/Fundraising.csv')

### Frequency table of donors VS non donors with the average donation for all donors. 

In [2]:
fund_df['TARGET_B'].value_counts() 

0    1560
1    1560
Name: TARGET_B, dtype: int64

In [3]:
#Average Donation for all donors
fund_df.groupby('TARGET_B')['TARGET_D'].mean()

TARGET_B
0     0.000000
1    12.999224
Name: TARGET_D, dtype: float64

In [4]:
fund_df[fund_df['TARGET_B']==1]['TARGET_D'].mean()

12.99922435897436

### Drop non-useful columns ['Row Id', 'Row Id.', 'TARGET_D']

In [5]:
fund_df.drop(['Row Id', 'Row Id.', 'TARGET_D'], axis=1, inplace=True)

### Partition the dataset into 60% training and 40% validation with specifying“TARGET_B”is the outcome variable, while others are input variables in form of X and Y objects.

In [6]:
predictors = ['zipconvert_2', 'zipconvert_3', 'zipconvert_4', 'zipconvert_5',
       'homeowner dummy', 'NUMCHLD', 'INCOME', 'gender dummy', 'WEALTH', 'HV',
       'Icmed', 'Icavg', 'IC15', 'NUMPROM', 'RAMNTALL', 'MAXRAMNT', 'LASTGIFT',
       'totalmonths', 'TIMELAG', 'AVGGIFT']
outcome = 'TARGET_B'

In [7]:
x= fund_df[predictors]
y=fund_df[outcome]

In [8]:
from sklearn.model_selection import train_test_split
train_x, valid_x, train_y, valid_y = train_test_split(x, y, test_size = 0.4, random_state = 1)

### Logistic Regression

In [9]:
from sklearn.linear_model import LogisticRegression
from dmba import plotDecisionTree, classificationSummary, regressionSummary

logreg = LogisticRegression()

In [10]:
logreg.fit(train_x, train_y)
classificationSummary(train_y, logreg.predict(train_x))
classificationSummary(valid_y, logreg.predict(valid_x))
LR_table = pd.crosstab(valid_y, logreg.predict(valid_x)) 

Confusion Matrix (Accuracy 0.5604)

       Prediction
Actual   0   1
     0 571 364
     1 459 478
Confusion Matrix (Accuracy 0.5497)

       Prediction
Actual   0   1
     0 381 244
     1 318 305


C:\Users\rs7am\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Classification Tree

In [11]:
from pathlib import Path

import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
import matplotlib.pylab as plt


In [12]:
fullClassTree = DecisionTreeClassifier()
fullClassTree.fit(train_x, train_y)
classificationSummary(train_y, fullClassTree.predict(train_x))
classificationSummary(valid_y, fullClassTree.predict(valid_x))

Confusion Matrix (Accuracy 1.0000)

       Prediction
Actual   0   1
     0 935   0
     1   0 937
Confusion Matrix (Accuracy 0.5088)

       Prediction
Actual   0   1
     0 305 320
     1 293 330


In [13]:
DT_table = pd.crosstab(valid_y, fullClassTree.predict(valid_x)) 

### Random Forest

In [14]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [15]:
rf = RandomForestClassifier(n_estimators=500, random_state=1)
rf.fit(train_x, train_y)
classificationSummary(train_y, rf.predict(train_x))
classificationSummary(valid_y, rf.predict(valid_x))
importances = rf.feature_importances_

Confusion Matrix (Accuracy 1.0000)

       Prediction
Actual   0   1
     0 935   0
     1   0 937
Confusion Matrix (Accuracy 0.5481)

       Prediction
Actual   0   1
     0 337 288
     1 276 347


In [16]:
RF_table = pd.crosstab(valid_y, rf.predict(valid_x)) 

In [17]:
df = pd.DataFrame({'feature':train_x.columns, 'importance': importances})
df = df.sort_values('importance')
print(df)

            feature  importance
5           NUMCHLD    0.006942
1      zipconvert_3    0.009962
2      zipconvert_4    0.010910
0      zipconvert_2    0.011482
3      zipconvert_5    0.012293
4   homeowner dummy    0.012966
7      gender dummy    0.014432
8            WEALTH    0.032616
6            INCOME    0.044016
15         MAXRAMNT    0.053709
16         LASTGIFT    0.054303
17      totalmonths    0.061583
18          TIMELAG    0.064141
12             IC15    0.074016
13          NUMPROM    0.081112
14         RAMNTALL    0.087535
10            Icmed    0.087732
11            Icavg    0.089925
19          AVGGIFT    0.094812
9                HV    0.095514


### Neural Network

In [18]:
from sklearn.neural_network import MLPClassifier

In [19]:
classes = sorted(y.unique())

In [20]:
clf = MLPClassifier(hidden_layer_sizes=(1), activation='logistic', solver='lbfgs', random_state=1)

In [21]:
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
train_x_NN = min_max_scaler.fit_transform(train_x)
valid_x_NN = min_max_scaler.fit_transform(valid_x)

In [22]:
clf.fit(train_x_NN, train_y)
clf.predict(valid_x_NN)

array([1, 1, 0, ..., 0, 1, 0], dtype=int64)

In [23]:
# Network structure
print('Intercepts')
print(clf.intercepts_)

print('weights')
print(clf.coefs_)

Intercepts
[array([-4.31281221]), array([-0.43011698])]
weights
[array([[ 3.03945487],
       [ 2.75481512],
       [ 2.93335671],
       [ 2.59244675],
       [ 0.90769653],
       [-0.64345292],
       [ 0.64086136],
       [ 0.07519465],
       [-0.1301672 ],
       [ 0.90303032],
       [ 7.39543893],
       [-7.00019201],
       [ 0.35324799],
       [-1.3852201 ],
       [ 2.28389426],
       [ 1.16432918],
       [-9.35776588],
       [-2.48426918],
       [ 2.74301159],
       [-5.92160441]]), array([[10.74906968]])]


In [24]:
# prediction

print(pd.concat([
    fund_df,
    pd.DataFrame(clf.predict_proba(valid_x_NN), columns=classes)
], axis=1)) 

      zipconvert_2  zipconvert_3  zipconvert_4  zipconvert_5  homeowner dummy  \
0                0             1             0             0                1   
1                1             0             0             0                1   
2                0             0             0             1                0   
3                0             0             0             1                1   
4                0             1             0             0                1   
...            ...           ...           ...           ...              ...   
3115             0             0             0             1                1   
3116             0             1             0             0                0   
3117             0             0             0             1                1   
3118             0             0             0             1                1   
3119             0             0             1             0                1   

      NUMCHLD  INCOME  gend

In [25]:
classificationSummary(train_y, clf.predict(train_x_NN), class_names=classes)
classificationSummary(valid_y, clf.predict(valid_x_NN), class_names=classes)
NN_table = pd.crosstab(valid_y, clf.predict(valid_x_NN))

Confusion Matrix (Accuracy 0.5582)

       Prediction
Actual   0   1
     0 631 304
     1 523 414
Confusion Matrix (Accuracy 0.5561)

       Prediction
Actual   0   1
     0 450 175
     1 379 244


### Linear Discriminant Analysis

In [26]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 

In [27]:
clf = LinearDiscriminantAnalysis()
clf.fit(train_x, train_y)
clf.predict(valid_x)

array([1, 1, 0, ..., 1, 1, 1], dtype=int64)

In [28]:
result_df = fund_df.copy()
result_df['Dec. Function'] = clf.decision_function(x)
result_df['Prediction'] = clf.predict(x)
result_df['p(Donor)'] = clf.predict_proba(x)[:, 1]
result_df

,zipconvert_2,zipconvert_3,zipconvert_4,zipconvert_5,homeowner dummy,NUMCHLD,INCOME,gender dummy,WEALTH,HV,...,RAMNTALL,MAXRAMNT,LASTGIFT,totalmonths,TIMELAG,AVGGIFT,TARGET_B,Dec. Function,Prediction,p(Donor)
0,0,1,0,0,1,1,5,1,9,1399,...,102.0,6.0,5.0,29,3,4.857143,1,0.276418,1,0.568668
1,1,0,0,0,1,1,1,0,7,698,...,94.0,12.0,12.0,34,6,9.400000,1,-0.368039,0,0.409015
2,0,0,0,1,0,2,5,1,8,828,...,30.0,10.0,5.0,29,7,4.285714,1,0.052670,1,0.513164
3,0,0,0,1,1,1,3,0,4,1471,...,177.0,10.0,8.0,30,3,7.080000,0,-0.057740,0,0.485569
4,0,1,0,0,1,1,4,0,8,547,...,23.0,11.0,11.0,30,6,7.666667,0,0.073872,1,0.518459
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3115,0,0,0,1,1,1,4,0,8,697,...,30.0,16.0,4.0,29,5,7.500000,0,0.171776,1,0.542839
3116,0,1,0,0,0,1,2,0,8,590,...,53.0,30.0,30.0,37,9,26.500000,1,-0.991284,0,0.270658
3117,0,0,0,1,1,1,3,0,7,3129,...,212.0,20.0,20.0,36,3,11.157895,0,-0.319825,0,0.420718
3118,0,0,0,1,1,1,7,1,8,1345,...,60.0,20.0,20.0,31,9,20.000000,0,0.156174,1,0.538964


In [29]:
classificationSummary(train_y, clf.predict(train_x), class_names=classes)
classificationSummary(valid_y, clf.predict(valid_x), class_names=classes)
LDA_table = pd.crosstab(y, clf.predict(x))

Confusion Matrix (Accuracy 0.5561)

       Prediction
Actual   0   1
     0 501 434
     1 397 540
Confusion Matrix (Accuracy 0.5633)

       Prediction
Actual   0   1
     0 340 285
     1 260 363


### Best model


In [30]:
def model_eval(matrix, model_name):
    tn = matrix.iloc[0,0] 
    tp = matrix.iloc[1,1] 
    fn = matrix.iloc[1,0] 
    fp = matrix.iloc[0,1] 
    tap = fn+tp
    tan = tn+fp
    tpn = tn+fn
    tpp = fp+tp
    precision = tp / tpp
    recall = tp / tap
    total = tn+tp+fn+fp
    
    
    data = [
        round((tp+tn)/total,4),
        round(1-((tp+tn)/total),4),
        round(tp /tap,4),
        round(tn /tan,4),
        round(precision,4),
                                    ]
    
    return(pd.DataFrame(data, columns=[model_name], 
                        index = ['Accuracy','Error Rate','Sensitivity','Specificity','Precision']))


In [31]:
LR = model_eval(LR_table , model_name = 'Logistic Regression')
CT = model_eval(DT_table , model_name = 'Classification Tree')
RF = model_eval(RF_table , model_name = 'Random Forest')
NN = model_eval(NN_table , model_name = 'Neural Network')
LDA = model_eval(LDA_table , model_name = 'Linear Discriminant Analysis')

pd.concat([LR, CT, RF, NN, LDA], axis=1)

,Logistic Regression,Classification Tree,Random Forest,Neural Network,Linear Discriminant Analysis
Accuracy,0.5497,0.5088,0.5481,0.5561,0.5590
Error Rate,0.4503,0.4912,0.4519,0.4439,0.4410
Sensitivity,0.4896,0.5297,0.5570,0.3917,0.5788
Specificity,0.6096,0.4880,0.5392,0.7200,0.5391
Precision,0.5556,0.5077,0.5465,0.5823,0.5567


<h6> The best model is the Linear Discriminant Analysis (LDA) as it has the highest accuracy and sensitivity in comparison to other models. Further, Sarkar (2019) mentioned the LDA is a more stable model than Logistic Regression in general leading to our decision to choose the LDA beside the performance of the model.
    
    Reference: https://www.knowledgehut.com/blog/data-science/linear-discriminant-analysis-for-machine-learning